In [15]:
#from datasets import load_dataset

In [16]:
#dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")

In [1]:
import pandas as pd
from transformers import BertTokenizer

c:\Program Files\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [6]:
df_yn = pd.read_csv("C:/Users/USER/삼성화재/git_share/hyper-personalized_car_accident_prediction/Dataset/고객별 사고 유무 기본 데이터.csv", encoding = "UTF-8", engine='python')

In [7]:
df_yn.dtypes

연령대           float64
성별            float64
국산차량여부        float64
직전3년간사고건수      object
차량경과년수         object
차종             object
운전자한정특별약관      object
가입경력코드        float64
차량가입금액         object
영상기록장치특약가입     object
마일리지약정거리       object
유효대수            int64
사고유무            int64
dtype: object

In [8]:
def remove_decimal_and_convert_to_str(value):
    if pd.isnull(value):
        return 'nan'
    if isinstance(value, float):
        value = int(value) if value.is_integer() else value
    return str(value)

In [9]:
df_yn['processed'] = df_yn.apply(lambda row: '/'.join(row.apply(remove_decimal_and_convert_to_str)), axis=1)


In [2]:
# df_yn['processed'] = df_yn.apply(lambda row: '/'.join(row.astype(str)), axis=1)

In [10]:
df_yn["processed"][1]

'0/1/nan/N/신차/기타/기명피보험자1인한정/8/미가입/미가입/15000K/1/0'

In [11]:
sentences = df_yn['processed'].tolist()

In [12]:
labels = [int(sentence.split('/')[-1]) for sentence in sentences]


In [12]:
%pip install torch torchvision torchaudio


  Using cached torch-2.3.0-cp310-cp310-win_amd64.whl.metadata (26 kB)
  Using cached torchvision-0.18.0-cp310-cp310-win_amd64.whl.metadata (6.6 kB)
  Using cached torchaudio-2.3.0-cp310-cp310-win_amd64.whl.metadata (6.4 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.2 kB)
  Using cached tbb-2021.12.0-py3-none-win_amd64.whl.metadata (1.1 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/159.8 MB ? eta -:--:--
   ---------------------------------------- 0.4/159.8 MB 10.9 MB/s eta 0:00:15
   ---------------------------------------- 0.7/159.8 MB 8.7 MB/s eta 0:00:19
   ---------------------------------------- 1.0/159.8 MB 7.8 MB/s eta 0:00:21
   ---------------------------------------- 1.3/159.8 MB 7.

In [20]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [14]:
%pip install tokenizers

Note: you may need to restart the kernel to use updated packages.


In [14]:
# 문장 리스트를 텍스트 파일로 저장
with open("sentences.txt", "w", encoding="utf-8") as file:
    for sentence in sentences:
        file.write(sentence + '\n')

In [29]:
from tokenizers import BertWordPieceTokenizer

In [30]:
# 새 WordPiece 토크나이저 초기화
tokenizer = BertWordPieceTokenizer(clean_text=True, handle_chinese_chars=True)

In [33]:
# 토크나이저 훈련
tokenizer.train(
    files="C:/Users/Minho Cho/KU_ML/hyper-personalized_car_accident_prediction/Model/BERT/sentences.txt",
    vocab_size=30522,
    min_frequency=2,
    show_progress=True,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    limit_alphabet=1000,
    wordpieces_prefix="##"
)

# 훈련된 토크나이저 저장
tokenizer.save_model(".", "bert-wordpiece")

['.\\bert-wordpiece-vocab.txt']

In [36]:
from tokenizers import BertWordPieceTokenizer

# 저장된 토크나이저 모델 로드
tokenizer = BertWordPieceTokenizer("bert-wordpiece-vocab.txt", lowercase=True)

# 테스트할 문장
test_sentences = df_yn["processed"][1]

# 각 문장을 토큰화하고 결과 출력
for sentence in test_sentences:
    output = tokenizer.encode(sentence)
    print(f"Original: {sentence}")
    print(f"Tokens: {output.tokens}")
    print(f"Token IDs: {output.ids}\n")


Original: 0
Tokens: ['[CLS]', '0', '[SEP]']
Token IDs: [2, 8, 3]

Original: /
Tokens: ['[CLS]', '/', '[SEP]']
Token IDs: [2, 7, 3]

Original: 1
Tokens: ['[CLS]', '1', '[SEP]']
Token IDs: [2, 9, 3]

Original: /
Tokens: ['[CLS]', '/', '[SEP]']
Token IDs: [2, 7, 3]

Original: n
Tokens: ['[CLS]', 'n', '[SEP]']
Token IDs: [2, 23, 3]

Original: a
Tokens: ['[CLS]', 'a', '[SEP]']
Token IDs: [2, 18, 3]

Original: n
Tokens: ['[CLS]', 'n', '[SEP]']
Token IDs: [2, 23, 3]

Original: /
Tokens: ['[CLS]', '/', '[SEP]']
Token IDs: [2, 7, 3]

Original: N
Tokens: ['[CLS]', 'n', '[SEP]']
Token IDs: [2, 23, 3]

Original: /
Tokens: ['[CLS]', '/', '[SEP]']
Token IDs: [2, 7, 3]

Original: 신
Tokens: ['[CLS]', '시', '##ᆫ', '[SEP]']
Token IDs: [2, 174, 68, 3]

Original: 차
Tokens: ['[CLS]', 'ᄎ', '##ᅡ', '[SEP]']
Token IDs: [2, 33, 63, 3]

Original: /
Tokens: ['[CLS]', '/', '[SEP]']
Token IDs: [2, 7, 3]

Original: 기
Tokens: ['[CLS]', '기', '[SEP]']
Token IDs: [2, 129, 3]

Original: 타
Tokens: ['[CLS]', 'ᄐ', '##ᅡ', '

In [83]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import SMOTE
import numpy as np

In [84]:
# Load the dataset
df_yn = pd.read_csv("C:/Users/Minho Cho/KU_ML/hyper-personalized_car_accident_prediction/Dataset/고객별 사고 유무 기본 데이터.csv", encoding="UTF-8", engine='python')


In [85]:
# Function to preprocess the data
def remove_decimal_and_convert_to_str(value):
    if pd.isnull(value):
        return 'nan'
    if isinstance(value, float):
        value = int(value) if value.is_integer() else value
    return str(value)

In [86]:
labels = df_yn.iloc[:, -1].tolist()
df_yn = df_yn.iloc[:, :-1]
df_yn['processed'] = df_yn.apply(lambda row: '/'.join(row.apply(remove_decimal_and_convert_to_str)), axis=1)


In [87]:
# Extract sentences
sentences = df_yn['processed'].tolist()

In [25]:
# toy_sentences, eval_sentences, toy_labels, eval_labels = train_test_split(sentences, labels, test_size=0., random_state=42)

In [88]:
# Split data into train and test sets
train_sentences, test_sentences, train_labels, test_labels = train_test_split(sentences, labels, test_size=0.2, random_state=42)


In [89]:
# Convert text data to numerical vectors for SMOTE
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_sentences, padding=True, truncation=True, return_tensors="np")
train_vectors = np.array(train_encodings['input_ids'])

In [90]:
# Apply SMOTE
smote = SMOTE(random_state=42)
train_vectors_resampled, train_labels_resampled = smote.fit_resample(train_vectors, train_labels)


In [91]:
# Convert resampled data back to the required format
train_encodings_resampled = {
    'input_ids': torch.tensor(train_vectors_resampled),
    'attention_mask': torch.tensor(np.ones_like(train_vectors_resampled))
}

In [92]:
# Tokenize the test sentences
test_encodings = tokenizer(test_sentences, padding=True, truncation=True, return_tensors="pt")


In [71]:
# # Initialize tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [72]:
# # Tokenize the sentences
# train_encodings = tokenizer(train_sentences, padding=True, truncation=True, return_tensors="pt")
# test_encodings = tokenizer(test_sentences, padding=True, truncation=True, return_tensors="pt")


In [93]:
# Create a Dataset class
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [94]:
# Create datasets
train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)


In [95]:
# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [96]:
# Load BERT model for binary classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [97]:
# Compute class weights to handle class imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)


In [98]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

C:\Users\Minho Cho\anaconda3\Lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [99]:
# Training the model
model.train()
for epoch in range(3):  # Train for 3 epochs
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        # Apply class weights to loss
        weighted_loss = loss * class_weights[labels].mean()
        weighted_loss.backward()
        optimizer.step()

In [100]:
# Save the model
model_save_path = "bert_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to bert_model.pth


In [101]:
# Evaluate the model
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).tolist())
        true_labels.extend(labels.tolist())


C:\Users\Minho Cho\AppData\Local\Temp\ipykernel_17736\2129374265.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [102]:
# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(true_labels)
print(predictions)

Accuracy: 0.7033572335499291
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
[0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,

C:\Users\Minho Cho\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [64]:
# Load the model (example usage)
model_load_path = "bert_model.pth"
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.load_state_dict(torch.load(model_load_path))
model.to(device)
print(f"Model loaded from {model_load_path}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded from bert_model.pth
